In [45]:
import torch
from torch import nn
from torchmetrics import Accuracy, F1Score, Precision, AUROC
import pytorch_lightning as pl
import torchsummary

from torchmetrics import Accuracy, F1Score, Precision, AUROC, ConfusionMatrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pandas as pd
from avalanche.benchmarks import benchmark_with_validation_stream, nc_benchmark
from torch.utils.data import Dataset

In [46]:




class TorchDataset(Dataset):

    def __init__(self,filePath):

        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Read CSV
        data = pd.read_csv(filePath)

        self.X = data.iloc[:,:-1].values
        self.targets = data.iloc[:, -1].values

        # Feature Scale if you want


        # Convert to Torch Tensors
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.targets = torch.tensor(self.targets, dtype=torch.int)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, item):

        return self.X[item], self.targets[item]


def prep_benchmark(train_loc, test_loc):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    hdata_train = TorchDataset(train_loc)
    hdata_test = TorchDataset(test_loc)

    return benchmark_with_validation_stream(nc_benchmark(train_dataset=hdata_train, test_dataset=hdata_test
                                 , shuffle=True, seed=1234, task_labels=False, per_exp_classes={0:7},n_experiences=7))

In [47]:
from avalanche.models import PNN
from avalanche.models.dynamic_modules import (
    MultiTaskModule,
    MultiHeadClassifier, IncrementalClassifier, DynamicModule,
)


class Simple1DCNN(nn.Module):
    """
    Convolutional Neural Network

    **Example**::

         from avalanche.models import SimpleCNN
         n_classes = 10 # e.g. MNIST
         model = SimpleCNN(num_classes=n_classes)
         print(model) # View model details
    """

    def __init__(self, num_classes=10):
        super(Simple1DCNN, self).__init__()

        self.features = nn.Sequential(
        nn.Conv1d(in_channels=1, out_channels=32, padding='same',  kernel_size=3,),
        nn.MaxPool1d(4),
        nn.Conv1d(in_channels=32, out_channels=32, padding='same',  kernel_size=3),
        nn.MaxPool1d(4),
        nn.Conv1d(in_channels=32, out_channels=16, padding='same',  kernel_size=3),
        nn.MaxPool1d(4),
        nn.Conv1d(in_channels=16, out_channels=16, padding='same',  kernel_size=3),
        nn.MaxPool1d(4),
        )
        self.classifier = nn.Sequential(nn.Linear(256, num_classes))


# IN MTS it doesn't even use this foward
    def forward(self, x):
        x = x.unsqueeze(dim=1)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        # x = self.classifier(x, task_labels)#TODO: previously

        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)
        return x



class MTSimple1DCNN(Simple1DCNN, DynamicModule):
    """
    Convolutional Neural Network
    with multi-head classifier
    """

    def __init__(self):
        super().__init__()
        self.classifier = IncrementalClassifier(in_features=256, initial_out_features=1,masking=False)

# CHECK: `initial_out_features`. is the problem her it has to be consistant, so we are not going to use a multitask module
    def forward(self, x, ):
        x = x.unsqueeze(dim=1)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        print(f'after squeeze : {x.shape}')
        x = self.classifier(x,)
        return x


In [48]:
# from avalanche.models import DynamicModule
# from avalanche.benchmarks import CLExperience
#
#
# class Incremental1DClassifier(DynamicModule):
#     """
#     Output layer that incrementally adds units whenever new classes are
#     encountered.
#
#     Typically used in class-incremental benchmarks where the number of
#     classes grows over time.
#     """
#
#     def __init__(
#         self,
#         in_features,
#         initial_out_features=2,
#         masking=True,
#         mask_value=-1000,
#     ):
#         """
#         :param in_features: number of input features.
#         :param initial_out_features: initial number of classes (can be
#             dynamically expanded).
#         :param masking: whether unused units should be masked (default=True).
#         :param mask_value: the value used for masked units (default=-1000).
#         """
#         super().__init__()
#         self.masking = masking
#         self.mask_value = mask_value
#
#         self.classifier = torch.nn.Linear(in_features, initial_out_features)
#         au_init = torch.zeros(initial_out_features, dtype=torch.bool)
#         self.register_buffer("active_units", au_init)
#
#
#     @torch.no_grad()
#     def adaptation(self, experience: CLExperience):
#         """If `dataset` contains unseen classes the classifier is expanded.
#
#         :param experience: data from the current experience.
#         :return:
#         """
#         in_features = self.classifier.in_features
#         old_nclasses = self.classifier.out_features
#         curr_classes = experience.classes_in_this_experience
#         new_nclasses = max(self.classifier.out_features, max(curr_classes) + 1)
#
#         # update active_units mask
#         if self.masking:
#             if old_nclasses != new_nclasses:  # expand active_units mask
#                 old_act_units = self.active_units
#                 self.active_units = torch.zeros(new_nclasses, dtype=torch.bool)
#                 self.active_units[: old_act_units.shape[0]] = old_act_units
#             # update with new active classes
#             if self.training:
#                 self.active_units[curr_classes] = 1
#
#         # update classifier weights
#         if old_nclasses == new_nclasses:
#             return
#         old_w, old_b = self.classifier.weight, self.classifier.bias
#         self.classifier = torch.nn.Linear(in_features, new_nclasses)
#         self.classifier.weight[:old_nclasses] = old_w
#         self.classifier.bias[:old_nclasses] = old_b
#
#     def forward(self, x, **kwargs):
#         """compute the output given the input `x`. This module does not use
#         the task label.
#
#         :param x:
#         :return:
#         """
#         out = self.classifier(x)
#         if self.masking:
#             out[..., torch.logical_not(self.active_units)] = self.mask_value
#         return out

In [49]:

class CNN1D(pl.LightningModule):
    def __init__(self, input_shape):

        self.save_hyperparameters()# to Save Hyperparameters
        super(CNN1D, self).__init__()
        self.conv1D_1 = nn.Conv1d(in_channels=1, out_channels=32, padding='same',  kernel_size=3,)
        self.maxPool1D_1 = nn.MaxPool1d(4)
        self.conv1D_2 = nn.Conv1d(in_channels=32, out_channels=32, padding='same',  kernel_size=3)
        self.maxPool1D_2 = nn.MaxPool1d(4)
        self.conv1D_3 = nn.Conv1d(in_channels=32, out_channels=16, padding='same',  kernel_size=3)
        self.maxPool1D_3 = nn.MaxPool1d(4)
        self.conv1D_4 = nn.Conv1d(in_channels=16, out_channels=16, padding='same',  kernel_size=3)
        self.maxPool1D_4 = nn.MaxPool1d(4)



        self.fc1 = nn.Linear(256, 300)
        self.fc2 = nn.Linear(300, 128)
        self.fc3 = nn.Linear(128, 31)

        self.train_accuracy = Accuracy(task='multiclass', num_classes=31)
        self.train_f1 = F1Score(task='multiclass', num_classes=31)
        self.train_precisn = Precision(task='multiclass', num_classes=31)
        self.train_rocAUC = AUROC(task='multiclass', num_classes=31)

        self.val_accuracy = Accuracy(task='multiclass', num_classes=31)
        self.val_f1 = F1Score(task='multiclass', num_classes=31)
        self.val_precisn = Precision(task='multiclass', num_classes=31)
        self.val_rocAUC = AUROC(task='multiclass', num_classes=31)

        self.test_accuracy = Accuracy(task='multiclass', num_classes=31)
        self.test_f1 = F1Score(task='multiclass', num_classes=31)
        self.test_precisn = Precision(task='multiclass', num_classes=31)
        self.test_rocAUC = AUROC(task='multiclass', num_classes=31)
        self.confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=31)


    def forward(self, x):
        # block 1
        x = x.unsqueeze(dim=1)
        x = self.conv1D_1(x)
        x = torch.relu(x)
        x= self.maxPool1D_1(x)


        # block 2
        x = self.conv1D_2(x)
        x = torch.relu(x)
        x= self.maxPool1D_2(x)

        # block 3
        x = self.conv1D_3(x)
        x = torch.relu(x)
        x= self.maxPool1D_3(x)

        # block 4
        x = self.conv1D_4(x)
        x = torch.relu(x)
        x= self.maxPool1D_4(x)

        # Flatten

        x = x.reshape(x.shape[0], -1)

        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.unsqueeze(dim=1)

        y_pred = self.forward(x)
        loss = nn.CrossEntropyLoss()(y_pred,y.long())
        self.log('train_loss', loss, on_epoch=True, on_step=True,prog_bar=True)
        self.log('train_acc_step', self.train_accuracy(y_pred, y.long(), ), on_epoch=True, on_step=True,prog_bar=False)
        self.log('train_f1_step', self.train_f1(y_pred, y.long(), ), on_epoch=True, on_step=True,prog_bar=False)
        self.log('train_precision_step', self.train_precisn(y_pred, y.long(), ), on_epoch=True, on_step=True,)
        self.log('train_auc_step', self.train_rocAUC(y_pred, y.long(), ), on_epoch=True, on_step=True,prog_bar=False )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.unsqueeze(dim=1)
        y_pred = self.forward(x)
        loss = nn.CrossEntropyLoss()(y_pred,y.long())
        self.log('validation_loss', loss)
        self.log('val_loss', loss, on_epoch=True)
        self.log('val_acc', self.val_accuracy(y_pred, y.long(), ),on_epoch=True, prog_bar=True)
        self.log('val_f1', self.val_f1(y_pred, y.long(), ),on_epoch=True, prog_bar=True)
        self.log('val_precision', self.val_precisn(y_pred, y.long(), ),on_epoch=True, prog_bar=True)
        self.log('val_auc', self.val_rocAUC(y_pred, y.long(), ),on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        x = x.unsqueeze(dim=1)
        # img = x.view(-1,3,IMG_SIZE,IMG_SIZE)
        y_pred = self.forward(x)
        loss = nn.CrossEntropyLoss()(y_pred,y.long())
        # self.log('test_loss', loss)
        # self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', self.test_accuracy(y_pred, y.long(), ),)
        self.log('test_f1', self.test_f1(y_pred, y.long(), ),)
        self.log('test_precision', self.test_precisn(y_pred, y.long(), ),)
        self.log('test_auc', self.test_rocAUC(y_pred, y.long(), ),)
        self.confusion_matrix(y_pred, y.long())
        print(self.confusion_matrix)
        # self.logger.experiment.add_confusion_matrix('confusion_matrix', self.confusion_matrix)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# model = CNN1D(input_shape=4097,)

In [50]:
from avalanche.models import MTSimpleMLP, MTSimpleCNN
from torch.nn import CrossEntropyLoss
from avalanche.training import Naive, EWC
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, timing_metrics, forgetting_metrics, \
    cpu_usage_metrics, disk_usage_metrics
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import TensorboardLogger, TextLogger, InteractiveLogger
import avalanche as avl




benchmark = prep_benchmark(train_loc='./DATA/TRAIN_DATA.csv', test_loc='./DATA/TEST_DATA.csv')


# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    # confusion_matrix_metrics(num_classes=benchmark['inc_bench'].n_classes, save_image=False,
    #                          stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (EWC)
# model2 = avl.models.PNN(in_features=4096,hidden_features_per_column=1000,num_layers=7,)
# model3 =  MTSimpleCNN()
model4 = MTSimple1DCNN()
# TODO: 4096 is too much i might need to pass thought the cnn first and then pass it to the pnn
cl_strategy = EWC(
    model4, torch.optim.Adam(model4.parameters(), lr=0.001,),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=10, eval_mb_size=100,
    evaluator=eval_plugin,ewc_lambda=0.4)

# TRAINING LOOP
print('Starting experiment...')
results = []
model_incs = []
classes_exp= []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    model_incs.append(model4.classifier)
    classes_exp.append(experience.classes_in_this_experience)
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

cpu
Starting experiment...
Start of experience:  0
Current Classes:  [8, 13, 12, 5]
-- >> Start of training phase << --
0it [00:00, ?it/s]after squeeze : torch.Size([67, 256])
100%|██████████| 1/1 [00:00<00:00,  5.91it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 328767.3535
	DiskUsage_MB/train_phase/train_stream/Task000 = 328767.3535
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6528
	Loss_MB/train_phase/train_stream/Task000 = 2.6528
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.1495
	Time_Epoch/train_phase/train_stream/Task000 = 0.1495
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0000
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
0it [00:00, ?it/s]after squeeze : torch.Size([67, 256])
100%|██████████| 1/1 [00:00<00:00, 13.79it/s]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 328768.4561
	DiskUsage_MB/train_phase/train_stream/Task000 = 328768.4561
	Loss_Epoch/train_phase/train_stream/Task000 = 2.5005
	Loss_MB/

In [51]:
print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
res = cl_strategy.eval(benchmark.test_stream)

Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
0it [00:00, ?it/s]after squeeze : torch.Size([70, 256])
100%|██████████| 1/1 [00:00<00:00, 30.22it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 15.2000
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 328887.7236
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.3429
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 3.4081
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
-- Starting eval on experience 1 (Task 0) from test stream --
0it [00:00, ?it/s]after squeeze : torch.Size([40, 256])
100%|██████████| 1/1 [00:00<00:00, 40.17it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 15.2000
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 328888.5820
	ExperienceForgetting/eval_phase/test_stre

In [52]:
d = eval_plugin.get_last_metrics()


In [53]:
model4.classifier.classifier


Linear(in_features=256, out_features=31, bias=True)

In [54]:
model_incs


[IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 ),
 IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 ),
 IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 ),
 IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 ),
 IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 ),
 IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 ),
 IncrementalClassifier(
   (classifier): Linear(in_features=256, out_features=31, bias=True)
 )]

In [55]:
classes_exp

[[8, 13, 12, 5], [16, 4], [9, 17], [1, 2], [11, 6], [0, 3, 7], [14, 15]]

In [56]:
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    targets = experience.dataset.targets
    print(set(targets))

Start of experience:  0
Current Classes:  [8, 13, 12, 5]
{8, 13, 12, 5}
Start of experience:  1
Current Classes:  [16, 4]
{16, 4}
Start of experience:  2
Current Classes:  [9, 17]
{9, 17}
Start of experience:  3
Current Classes:  [1, 2]
{1, 2}
Start of experience:  4
Current Classes:  [11, 6]
{11, 6}
Start of experience:  5
Current Classes:  [0, 3, 7]
{0, 3, 7}
Start of experience:  6
Current Classes:  [14, 15]
{14, 15}
